In [ ]:
import os
import sys

import numpy as np
import pandas as pd

sys.path.append('../')
from utils import read_csv_time_chunks
from steps.base import BaseTransformer

In [ ]:
files_dir = '/mnt/ml-team/minerva/talking_data/files'
hours = list(range(12))
days = [7]

In [ ]:
df_train = read_csv_time_chunks(files_dir, days=days, hours=hours)

In [ ]:
blacklist = {'ip': [299172, 144604, 135992, 49386, 151908],
             'app': [151, 56, 183, 93],
             'device': [5, 182, 1728],
             'os': [56, 65, 39, 79, 97],
             'channel': [404, 420, 474]
             }

In [ ]:
class Blacklist(BaseTransformer):

    def __init__(self, blacklists={}):
        self.blacklist = blacklist
    
    @property
    def blacklist_names(self):
        blacklist_names = ['{}_on_blacklist'.format(category)
                                 for category in self.blacklist]
        return blacklist_names
    
    def transform(self, categorical_features):
        new_features = []

        for category, blacklist_name in zip(self.blacklist, self.blacklist_names):

                categorical_features[blacklist_name] = (categorical_features[category].isin(blacklist[category])).astype(int)
            
        return {'categorical_features': categorical_features[self.blacklist_names]}


In [ ]:
black = Blacklist(blacklist)

In [ ]:
is_black = black.transform(df_train)['categorical_features']

In [ ]:
is_black.head()

In [ ]:
is_black.loc[is_black['channel_on_blacklist'] == 1]